# Project : portfolio trading
by Janvier Nils, Jeulin Marc and Romano Julien

### Import librairies

In [1]:
import pandas as pd
import glob,os
import numpy as np
import timeit
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots


###  Reading the data

In [2]:
#path = os.getcwd()
extension = 'csv'

def getFiles():
    return glob.glob('chunk*.{}'.format(extension))

def prepareData():
    files = getFiles()
    list_of_df = []
    for file in files :
        data = pd.read_csv(file, index_col='Unnamed: 0')
        list_of_df.append(data)
    df = pd.concat(x for x in list_of_df)
    return df
df = prepareData()
df = df.sort_values('date')
df['market_cap']=df['PRC']*df['SHROUT']
df = df.drop(columns= {'TICKER', 'SHRCD', 'EXCHCD', 'SHROUT', 'VOL'})

In [3]:
df

,date,PERMNO,PRC,RET,market_cap
0,1962-07,10006,67.50000,0.144068,9.807750e+04
1379,1962-07,31085,-8.25000,NaN,-4.603500e+03
1378,1962-07,31077,11.25000,NaN,2.484000e+04
1377,1962-07,31069,-5.68750,NaN,-3.367000e+03
1376,1962-07,31050,3.12500,NaN,2.131250e+03
...,...,...,...,...,...
4218120,2019-12,52708,55.79000,-0.064711,1.561456e+07
4218119,2019-12,52695,338.51999,0.068055,1.823472e+07
4218118,2019-12,52679,17.76000,0.051593,1.546363e+05
4218130,2019-12,53373,8.65000,-0.187030,2.349081e+05


## Indexing by stocks

In [177]:
companies = df.PERMNO.unique()
list_of_returns = []
count = 0
for x in companies :
    a = df[df.PERMNO == x]
    annual_return = a.PRC.pct_change(12)
    annual_return = annual_return.shift(1)
    a.loc[:,"annual_return"]= annual_return
    list_of_returns.append(a)
    count = count +1
    if (count % 1000) == 0:
        print(count)
    
df = pd.concat(list_of_returns)
    

C:\Users\Admin\Anaconda3\envs\prod\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Admin\Anaconda3\envs\prod\lib\site-packages\pandas\core\indexing.py:964: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000


In [7]:
df = pd.read_pickle('donnees_processed.pkl')

In [8]:
df

,date,PERMNO,SHRCD,PRC,RET,market_cap,annual_return
0,1962-07-01,10006,10.0,67.5000,0.144068,98077.500,NaN
2057,1962-08-01,10006,10.0,67.1250,0.003704,97532.625,NaN
4122,1962-09-01,10006,10.0,63.5000,-0.054004,92265.500,NaN
6188,1962-10-01,10006,10.0,68.0000,0.070866,98804.000,NaN
8255,1962-11-01,10006,10.0,72.8750,0.080882,105887.375,NaN
...,...,...,...,...,...,...,...
4217747,2019-12-01,19122,73.0,20.1000,NaN,12060.000,NaN
4210080,2019-11-01,19121,NaN,NaN,NaN,NaN,NaN
4217746,2019-12-01,19121,73.0,20.0997,NaN,34169.490,NaN
4210078,2019-11-01,19119,NaN,NaN,NaN,NaN,NaN


In [10]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df[df.PERMNO == 31077].index,
               y=df[df.PERMNO == 31077]['annual_return'],
               name="annual_return"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df[df.PERMNO == 31077].index,
               y=df[df.PERMNO == 31077]['PRC'],
               name="price"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="annual_return vs Price"
)

# Set x-axis title
fig.update_xaxes(title_text="time")

# Set y-axes titles
fig.update_yaxes(title_text="<b>annual_return</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>price</b> of the index", secondary_y=True)

fig.show()

## Question 6 :
Using one of the six strategies described above, each group will create a monthly
trading strategy and form 10 portfolios (both equally- and value-weighted).


In [11]:
df['date'] = pd.to_datetime(df.date)
df['decile'] = df.groupby('date')['annual_return'].rank(pct=True, ascending=False)
df['decile'] = np.trunc(df['decile'] * 10) + 1
df['decile'] = np.where(df['decile'] == 11.0, 10, df['decile'])
df = df.dropna()
df.loc[df.date == "2019-09-01"].sort_values('annual_return', ascending=False)

,date,PERMNO,SHRCD,PRC,RET,market_cap,annual_return,decile
4191981,2019-09-01,14660,12.0,3.840,0.337979,28727.040,76.150538,1.0
4197430,2019-09-01,91021,11.0,3.420,-0.777053,11775.060,28.386973,1.0
4191605,2019-09-01,13965,11.0,4.990,0.054968,85179.300,23.256410,1.0
4192080,2019-09-01,14826,12.0,26.710,-0.042309,270144.940,21.064873,1.0
4197862,2019-09-01,92131,12.0,1.470,-0.261307,84930.720,18.586614,1.0
...,...,...,...,...,...,...,...,...
4194983,2019-09-01,46923,11.0,115.000,0.036036,231035.000,-2.470686,10.0
4190986,2019-09-01,12492,31.0,3.630,-0.103704,8156.610,-2.629779,10.0
4196212,2019-09-01,84419,12.0,3.260,-0.489028,14278.800,-2.692308,10.0
4194828,2019-09-01,29058,11.0,-9.255,0.030050,-34678.485,-2.811492,10.0


In [12]:
df.loc[df.date == "2019-01-01"]['annual_return'].max()

60.589743589743584

## Portfolio equally weighted

In [13]:
df1 = df.copy()

In [14]:
ptf_equ = df1.groupby(['date', 'decile'])[['annual_return']].agg(np.mean).rename(columns={'annual_return':'AR_mean'})
ptf_equ

AR_mean
date       decile          
1963-08-01 1.0     0.750325
           2.0     0.332660
           3.0     0.219412
           4.0     0.145600
           5.0     0.075956
...                     ...
2019-12-01 6.0     0.023659
           7.0    -0.040990
           8.0    -0.165794
           9.0    -0.375834
           10.0   -1.276089

[6770 rows x 1 columns]

In [15]:
df1['temp'] = list(zip(df1.date, df1.decile))
df1['AR_mean'] = df1.temp.map(dict(ptf_equ['AR_mean']))
df1.drop(columns=['temp'], inplace=True)

In [16]:
#df1['strat_momentum'] = np.where(df1['AR_mean']>=0, df1['RET'], -1*df1['RET'])

In [17]:
df1

,date,PERMNO,SHRCD,PRC,RET,market_cap,annual_return,decile,AR_mean
26967,1963-08-01,10006,10.0,103.00,0.121081,149659.00,0.370370,2.0,0.332660
29055,1963-09-01,10006,10.0,47.50,-0.077670,138035.00,0.534451,1.0,0.773501
31146,1963-10-01,10006,10.0,50.25,0.057895,146026.50,-0.251969,8.0,-0.235396
33245,1963-11-01,10006,10.0,62.25,0.246766,180898.50,-0.261029,9.0,-0.866874
35346,1963-12-01,10006,10.0,62.00,-0.004016,182776.00,-0.145798,8.0,-0.212265
...,...,...,...,...,...,...,...,...,...
4217133,2019-12-01,18253,11.0,1.93,0.245161,146238.03,-0.432234,9.0,-0.375834
4217132,2019-12-01,18250,12.0,-10.49,0.015489,-150793.75,-2.064948,10.0,-1.276089
4217131,2019-12-01,18249,12.0,10.35,0.002907,152041.50,0.061728,5.0,0.065967
4217130,2019-12-01,18248,12.0,-10.29,0.008824,-452760.00,0.059190,5.0,0.065967


In [188]:
# ptf_equ = pd.DataFrame({'strat_momentum' : df1.groupby(['date', 'decile'])['strat_momemtum'].agg(np.mean).reset_index()})
ptf_equ2['investment'] = df1.groupby(['date', 'decile'])[['RET']].agg(np.mean).rename(columns={'strat_momemtum':'investment'})


AR_mean  investment
date       decile                      
1963-08-01 1.0     0.750325    0.074975
           2.0     0.332660    0.056726
           3.0     0.219412    0.044586
           4.0     0.145600    0.042321
           5.0     0.075956    0.037225
...                     ...         ...
2019-12-01 6.0     0.023659    0.031604
           7.0    -0.040990    0.035139
           8.0    -0.165794    0.036695
           9.0    -0.375834    0.065249
           10.0   -1.276089    0.105347

[6770 rows x 2 columns]

In [18]:
print(f"return moyen total, sur 60 ans, moyen de l'ensemble des portefeuilles : {100*((1+ptf_equ.investment.mean())**(60*12)-1):.2f} %")

AttributeError: 'DataFrame' object has no attribute 'investment'

In [ ]:
ptf_equ.groupby('decile')[["investment"]].mean()

In [ ]:
# faire un bar chart avec les returns moyen par année a la place de ça :
# fig = go.Figure()
# fig.add_trace(
#     go.Scatter(x=pd.Series(ar.index.format()),
#                y=ar,
#                name="annual_return")
# )
# fig.show()
import plotly.express as px

fig = px.bar(x=ptf_equ.groupby('decile')[["investment"]].mean().index, y=ptf_equ.groupby('decile')[["investment"]].mean(), labels={'x':'total_bill', 'y':'count'})
fig.show()

## Portfolio value weighted

In [ ]:
total_mc = df1.market_cap.sum()
print(f'toatal market capitalisation total_mc = {total_mc}')
#df1['strat_momemtum_vw'] = np.where(df1['AR_mean']>=0, df1['RET'], -1*df1['RET'])
df1['strat_momemtum_vw'] = df1['RET']*df1['market_cap']/total_mc
df1

In [143]:
ptf_value_weighted =  df1.groupby(['date', 'decile'])[['strat_momemtum_vw']].agg(np.mean).reset_index()
ptf_value_weighted

,date,decile,strat_momemtum_vw
0,1963-08-01,1.0,1.847507e-09
1,1963-08-01,2.0,4.404964e-09
2,1963-08-01,3.0,2.447888e-09
3,1963-08-01,4.0,2.809344e-09
4,1963-08-01,5.0,2.691104e-09
...,...,...,...
6765,2019-12-01,6.0,1.850988e-08
6766,2019-12-01,7.0,-3.514403e-08
6767,2019-12-01,8.0,-2.221758e-08
6768,2019-12-01,9.0,-2.162015e-08


## Question 7 :
Finally, the raw return, CAPM alpha, Fama-French 3-factor alpha and Fama-French 5-
factor alpha of the 10 portfolios as well as the arbitrage portfolio that is long in
portfolio 10 and short in portfolio 1 will be reported.

The Formula for the Fama French Model Is:
\begin{aligned} &R_{it} - R_{ft} = \alpha_{it} + \beta_1 ( R_{Mt} - R_{ft} ) + \beta_2SMB_t + \beta_3HML_t + \epsilon_{it} \\ &\textbf{where:} \\ &R_{it} = \text{total return of a stock or portfolio } i \text{ at time } t \\ &R_{ft} = \text{risk free rate of return at time } t \\ &R_{Mt} = \text{total market portfolio return at time } t \\ &R_{it} - R_{ft} = \text{expected excess return} \\ &R_{Mt} - R_{ft} = \text{excess return on the market portfolio (index)} \\ &SMB_t = \text{size premium (small minus big)} \\ &HML_t = \text{value premium (high minus low)} \\ &\beta_{1,2,3} = \text{factor coefficients} \\ \end{aligned} 


__Note__ : voir avec https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_5_factors_2x3.html   
si l'on doit retirer les shares codes différents de 10 et 11

In [23]:
import statsmodels.formula.api as sm # module for stats models
from statsmodels.iolib.summary2 import summary_col # module for presenting stats models outputs nicely

__Fama-French 5 factors data :__

In [71]:
fama_french_5 = pd.read_csv("F-F_Research_Data_5_Factors_2x3.CSV", index_col='Unnamed: 0', skiprows=3)
fama_french_5 = fama_french_5[:680]
fama_french_5.index = pd.to_datetime(fama_french_5.index, format="%Y%m")
fama_french_5

,Mkt-RF,SMB,HML,RMW,CMA,RF
1963-07-01,-0.39,-0.47,-0.83,0.66,-1.15,0.27
1963-08-01,5.07,-0.79,1.67,0.39,-0.40,0.25
1963-09-01,-1.57,-0.48,0.18,-0.76,0.24,0.27
1963-10-01,2.53,-1.29,-0.10,2.75,-2.24,0.29
1963-11-01,-0.85,-0.84,1.71,-0.45,2.22,0.27
...,...,...,...,...,...,...
2019-10-01,2.06,0.21,-2.07,0.43,-0.96,0.15
2019-11-01,3.87,0.50,-1.86,-1.50,-1.29,0.12
2019-12-01,2.77,0.96,1.83,0.21,1.31,0.14
2020-01-01,-0.11,-4.40,-6.27,-1.23,-2.34,0.13


In [73]:
def assetPriceReg(df_stk):
    
    fama_french_5 = pd.read_csv("F-F_Research_Data_5_Factors_2x3.CSV", index_col='Unnamed: 0', skiprows=3)
    fama_french_5 = fama_french_5[:680]
    fama_french_5 = fama_french_5[fama_french_5.columns].apply(pd.to_numeric, errors='coerce')
    fama_french_5.index = pd.to_datetime(fama_french_5.index, format="%Y%m")
    
    
    # Reading in factor data
    
    fama_french_5.rename(columns={'Mkt-RF': 'MKT-RF'}, inplace=True)
    fama_french_5['MKT-RF'] = fama_french_5['MKT-RF']/100
    fama_french_5['SMB'] = fama_french_5['SMB']/100
    fama_french_5['HML'] = fama_french_5['HML']/100
    fama_french_5['RMW'] = fama_french_5['RMW']/100
    fama_french_5['CMA'] = fama_french_5['CMA']/100
    fama_french_5['RF']  = fama_french_5['RF']/100
    fama_french_5['MKT'] = fama_french_5['MKT-RF'] + fama_french_5['RF']
    
    # Merging the stock and factor returns dataframes together
    df_stock_factor = pd.merge(df_stk,fama_french_5,
                               left_on="date",
                               right_on=fama_french_5.index,
                               how='left').set_index('date', drop = True)
    df_stock_factor['XsRet'] = df_stock_factor['investment'] - df_stock_factor['RF'] # Calculating excess returns

    # Running CAPM, FF3, and FF5 models.
    CAPM = sm.ols(formula = 'XsRet ~ MKT', data=df_stock_factor).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    FF3 = sm.ols( formula = 'XsRet ~ MKT + SMB + HML', data=df_stock_factor).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    FF5 = sm.ols( formula = 'XsRet ~ MKT + SMB + HML + RMW + CMA', data=df_stock_factor).fit(cov_type='HAC',cov_kwds={'maxlags':1})

    CAPMtstat = CAPM.tvalues
    FF3tstat = FF3.tvalues
    FF5tstat = FF5.tvalues

    CAPMcoeff = CAPM.params
    FF3coeff = FF3.params
    FF5coeff = FF5.params

    # DataFrame with coefficients and t-stats
    results_df = pd.DataFrame({'CAPMcoeff':CAPMcoeff,'CAPMtstat':CAPMtstat,
                               'FF3coeff':FF3coeff, 'FF3tstat':FF3tstat,
                               'FF5coeff':FF5coeff, 'FF5tstat':FF5tstat},
    index = ['Intercept', 'MKT', 'SMB', 'HML', 'RMW', 'CMA'])


    dfoutput = summary_col([CAPM,FF3, FF5],stars=True,float_format='%0.4f',
                  model_names=['CAPM','FF3','FF5'],
                  info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                             'Adjusted R2':lambda x: "{:.4f}".format(x.rsquared_adj)}, 
                             regressor_order = ['Intercept', 'MKT', 'SMB', 'HML', 'RMW', 'CMA'])

    print(dfoutput)
    
    return results_df

In [92]:
def test(liste, ptf_equ, ind):
    print(ind)
    df = ptf_equ[ptf_equ["decile"] == ind]
    return liste.append(assetPriceReg(df))

#results_df = pd.DataFrame(deciles_list).apply(lambda x : test(liste,ptf_equ, x))
deciles_list = ptf_equ.decile.unique()
results_ew = []
for i in deciles_list:
    results_ew.append(assetPriceReg(ptf_equ[ptf_equ["decile"] == i]))


               CAPM      FF3        FF5    
-------------------------------------------
Intercept   0.0007    0.0012     0.0020    
            (0.0018)  (0.0012)   (0.0013)  
MKT         1.1795*** 0.9320***  0.9315*** 
            (0.0460)  (0.0328)   (0.0321)  
SMB                   1.0988***  1.0199*** 
                      (0.0801)   (0.0619)  
HML                   -0.2476*** -0.3536***
                      (0.0725)   (0.0781)  
RMW                              -0.3583***
                                 (0.0972)  
CMA                              0.2390**  
                                 (0.1156)  
R-squared   0.5904    0.8241     0.8396    
            0.5911    0.8249     0.8408    
N           677       677        677       
Adjusted R2 0.5904    0.8241     0.8396    
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01

               CAPM      FF3       FF5   
-----------------------------------------
Intercept   0.0027    0.0029*   0.0021   
            (0.0018) 


               CAPM       FF3        FF5    
--------------------------------------------
Intercept   -0.0089*** -0.0070*** -0.0089***
            (0.0020)   (0.0017)   (0.0023)  
MKT         -1.1407*** -0.9812*** -0.9237***
            (0.0672)   (0.0585)   (0.0635)  
SMB                    -1.0788*** -1.0159***
                       (0.1131)   (0.1029)  
HML                    -0.2840**  -0.4532***
                       (0.1321)   (0.1666)  
RMW                               0.2841*   
                                  (0.1578)  
CMA                               0.3688    
                                  (0.2688)  
R-squared   0.4514     0.6385     0.6459    
            0.4522     0.6401     0.6485    
N           677        677        677       
Adjusted R2 0.4514     0.6385     0.6459    
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


In [93]:
liste

[           CAPMcoeff  CAPMtstat  FF3coeff   FF3tstat  FF5coeff   FF5tstat
 Intercept   0.000688   0.381921  0.001211   1.016709  0.001985   1.565675
 MKT         1.179469  25.658622  0.932006  28.436905  0.931517  28.985408
 SMB              NaN        NaN  1.098768  13.714746  1.019934  16.489648
 HML              NaN        NaN -0.247596  -3.414493 -0.353565  -4.526722
 RMW              NaN        NaN       NaN        NaN -0.358274  -3.685965
 CMA              NaN        NaN       NaN        NaN  0.238998   2.068134,
            CAPMcoeff  CAPMtstat  FF3coeff  FF3tstat  FF5coeff   FF5tstat
 Intercept   0.002718   1.500551  0.002862  1.799710  0.002112   1.360814
 MKT         0.795937  10.230786  0.649127  8.837529  0.692037  10.225986
 SMB              NaN        NaN  0.679790  6.755155  0.657000   6.459446
 HML              NaN        NaN -0.113483 -0.981581 -0.335257  -2.078677
 RMW              NaN        NaN       NaN       NaN -0.104760  -1.294912
 CMA              NaN        N

In [76]:
results_df = assetPriceReg(ptf_equ[ptf_equ.decile ==10])


               CAPM       FF3        FF5    
--------------------------------------------
Intercept   -0.0089*** -0.0070*** -0.0089***
            (0.0020)   (0.0017)   (0.0023)  
MKT         -1.1407*** -0.9812*** -0.9237***
            (0.0672)   (0.0585)   (0.0635)  
SMB                    -1.0788*** -1.0159***
                       (0.1131)   (0.1029)  
HML                    -0.2840**  -0.4532***
                       (0.1321)   (0.1666)  
RMW                               0.2841*   
                                  (0.1578)  
CMA                               0.3688    
                                  (0.2688)  
R-squared   0.4514     0.6385     0.6459    
            0.4522     0.6401     0.6485    
N           677        677        677       
Adjusted R2 0.4514     0.6385     0.6459    
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


In [145]:
results_vw = []
for i in deciles_list:
    results_vw.append(assetPriceReg(ptf_value_weighted[ptf_value_weighted["decile"] == i]))

KeyError: 'investment'